In [1]:
!pip install langchain-community langchain-text-splitters ollama langchain-ollama  pypdf  langchain-huggingface sentence-transformers faiss-cpu PyMuPDF   --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 74.4 MB/s eta 0:00:00


In [2]:
# 第一個 Cell
!pip install -q bitsandbytes langchain-huggingface sentencepiece
!pip install -q -U transformers accelerate
# 注意：這裡刻意不寫 torch，直接用 Colab 內建的，就不會打架也不會慢

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 23.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 27.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [3]:
import os
import subprocess
import time
from google.colab import drive


# --- 2. 修正並安裝 Ollama ---
print("正在安裝相依套件 zstd...")
!sudo apt-get install -y zstd  # <--- 新增這一行解決你的錯誤

print("正在安裝 Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# --- 3. 啟動 Ollama 服務 ---
print("正在啟動 Ollama 服務...")
# 使用完整路徑以防萬一 (通常是 /usr/local/bin/ollama)
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# 等待服務啟動
print("等待服務啟動中 (約 10 秒)...")
time.sleep(10)

# --- 4. 測試連線 ---
try:
    # 檢查服務是否活著
    check = subprocess.run(["curl", "-s", "http://localhost:11434"], capture_output=True, text=True)
    if "Ollama is running" in check.stdout:
        print("✅ 成功：Ollama 服務已在背景執行！")
    else:
        print("⚠️ 警告：服務似乎未回應，請檢查日誌。")
except Exception as e:
    print(f"檢查連線時發生錯誤: {e}")

正在安裝相依套件 zstd...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (499 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package zstd.
(Reading database ... 121689 files and direct

In [4]:
import os
from google.colab import drive

# 1. 掛載 Google Drive
drive.mount('/content/drive')

# 2. 設定模型儲存路徑 (建議在 Drive 建立一個專門的資料夾)
# 這裡設定為 MyDrive 下的 RAG_project/ollama_models 資料夾
my_model_folder = '/content/drive/MyDrive/RAG/ollama_models'

# 如果資料夾不存在，建立它
if not os.path.exists(my_model_folder):
    os.makedirs(my_model_folder)

# 3. 【關鍵】設定環境變數，讓 Ollama 知道去哪裡找模型
os.environ['OLLAMA_MODELS'] = my_model_folder

print(f"Ollama 模型路徑已設定為: {os.environ['OLLAMA_MODELS']}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ollama 模型路徑已設定為: /content/drive/MyDrive/RAG/ollama_models


In [23]:
from langchain_ollama import ChatOllama
llm=ChatOllama(model="qwen3:4b-instruct-2507-q8_0",temperature=0)

In [6]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={'device': 'cuda'},
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [7]:
from pydantic import BaseModel,Field
from langchain_core.prompts import PromptTemplate
from langchain_core.retrievers import BaseRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from typing import List
from langchain_core.documents import Document

##Define the query classifer class

In [56]:
class categories_options(BaseModel):
    category: str = Field(description="The category of the query, the options are: Factual, Analytical, Opinion, or Contextual", example="Factual")

class QueryClassifier:
    def __init__(self):
        self.prompt=PromptTemplate(
            input_variables=["query"],
            template="Classify the following query into one of these categories: Factual, Analytical, Opinion, or Contextual.\nQuery: {query}\nCategory:"
        )
        self.chain=self.prompt | llm.with_structured_output(categories_options)

    def classify(self,query):
        print("query classify")
        ans=self.chain.invoke(query).category
        print(ans)
        return ans


/tmp/ipython-input-2567057734.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  category: str = Field(description="The category of the query, the options are: Factual, Analytical, Opinion, or Contextual", example="Factual")


##Define the Base Retriever class, such that the complex ones will inherit from it

In [57]:
class BaseRetrievalStrategy:
    def __init__(self,texts,embeddings,llm):
        self.embeddings=embeddings
        self.llm=llm
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=128)
        self.documents=text_splitter.create_documents(texts)
        self.bd=FAISS.from_documents(self.documents,self.embeddings)
    def retrieve(self,query,k=3):
        return self.bd.similarity_search(query,k=k)




##Define Factual retriever strategy

In [58]:
class relevant_score(BaseModel):
    score : float =Field(description="The relevance score of the document to the query", example=8.0)

class FactualRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, texts, embeddings, llm):
      super().__init__(texts, embeddings, llm)
    def retrieve(self,query,k=3):
        print("retrieving factual")

        # Use LLM to enhance the query
        enhance_query_prompt=PromptTemplate(
            input_variables=["query"],
            template="Enhance this factual query for better information retrieval: {query}"
        )
        query_chain= enhance_query_prompt | self.llm
        enhance_query=query_chain.invoke(query).content
        print(f"enchance query:{enhance_query}")

        # Retrieve documents using the enhanced query
        docs=self.bd.similarity_search(enhance_query,k=2)

        # Use LLM to rank the relevance of retrieved documents
        rank_prompt=PromptTemplate(
            input_variables=["query","doc"],
            template="On a scale of 1-10, how relevant is this document to the query: '{query}'?\nDocument: {doc}\nRelevance score:"
        )

        rank_chain=rank_prompt | self.llm.with_structured_output(relevant_score)
        ranked_docs=[]
        print("ranking docs")

        for doc in docs:
            input_data={"query":query,"doc":doc.page_content}
            score = float(rank_chain.invoke(input_data).score)
            ranked_docs.append((doc,score))

        # Sort by relevance score and return
        ranked_docs.sort(key=lambda x : x[1],reverse=True)
        return [doc for doc, _  in ranked_docs]



/tmp/ipython-input-288253709.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  score : float =Field(description="The relevance score of the document to the query", example=8.0)


##Define Analytical reriever strategy

In [59]:
class SelectedIndices(BaseModel):
    indices : List[int] = Field(description="Indices of selected documents", example=[0, 1, 2, 3])
class SubQueries(BaseModel):
    sub_queries : List[str] = Field(description= "List of sub-queries for comprehensive analysis", example=["What is the population of New York?", "What is the GDP of New York?"])

class AnalyticalRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, texts, embeddings, llm):
      super().__init__(texts, embeddings, llm)
    def retrive(self,query,k=3):
        print("retrieving analytical")

        # Use LLM to generate sub-queries for comprehensive analysis
        sub_queries_prompt = PromptTemplate(
            input_variables=["query","k"],
            template="Generate {k} sub-questions for: {query}"
        )
        sub_queries_chain = sub_queries_prompt | self.llm.with_structured_output(SubQueries)
        input_data={"query":query,"k":k}
        sub_queries=sub_queries_chain.invoke(input_data).sub_queries
        print(f'sub queries for comprehensive analysis: {sub_queries}')

        all_docs=[]
        for query in sub_queries:
            all_docs.extend(self.bd.similarity_search(query,k=2))

        # Use LLM to ensure diversity and relevance
        diversity_prompt=PromptTemplate(
            input_variables=["query","docs","k"],
            template="""Select the most diverse and relevant set of {k} documents for the query: '{query}'\nDocuments: {docs}\n
            Return only the indices of selected documents as a list of integers."""
        )

        diversity_chain=diversity_prompt | self.llm.with_structured_output(SelectedIndices)
        docs_text="\n".join([f"{i}: {doc.page_content[:100]}" for i ,doc in enumerate(all_docs)])
        input_data={"query":query,"docs":docs_text,"k":k}
        selected_indices_result=diversity_chain.invoke(input_data).indices
        print(f'selected diverse and relevant documents')

        return [all_docs[i] for i in selected_indices_result if i < len(all_docs)]


/tmp/ipython-input-885214608.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  indices : List[int] = Field(description="Indices of selected documents", example=[0, 1, 2, 3])
/tmp/ipython-input-885214608.py:4: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  sub_queries : List[str] = Field(description= "List of sub-queries for comprehensive analysis", example=["What is the population of New York?", "What is the GDP of New York?"])


##Define Opinion retriever strategy

In [60]:
class OpinionRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, texts, embeddings, llm):
      super().__init__(texts, embeddings, llm)
    def retrieve(self,query,k=3):
        print("retrieving opinion")
        # Use LLM to identify potential viewpoints
        viewpoint_prompt=PromptTemplate(
            input_variables=["query","k"],
            template="Identify {k} distinct viewpoints or perspectives on the topic: {query}"
        )
        viewpoint_chain=viewpoint_prompt | self.llm
        input_data={"query":query,"k":k}
        viewpoints=viewpoint_chain.invoke(input_data).content.split("\n")
        print(f'viewpoints: {viewpoints}')

        all_docs=[]
        for viewpoint in viewpoints:
            all_docs.extend(self.bd.similarity_search(f"{query} {viewpoint}",k=2))

        # Use LLM to classify and select diverse opinions
        opinion_prompt=PromptTemplate(
            input_variables=["query","docs","k"],
            template="Classify these documents into distinct opinions on '{query}' and select the {k} most representative and diverse viewpoints:\nDocuments: {docs}\nSelected indices:"
        )
        opinion_chain = opinion_prompt | self.llm.with_structured_output(SelectedIndices)

        docs_text="\n".join([f"{i} : {doc.page_content[:100]}" for i, doc in enumerate(all_docs)])
        input_data={"query":query,"doc":docs_text,"k":k}
        selected_indices =opinion_chain.invoke(input_data).indices
        print(f'selected diverse and relevant documents')

        return [all_docs[int((i))] for i in selected_indices.split() if i.isdigit() and int(i) < len(all_docs)]




##Define Contextual retriever strategy

In [61]:
class ContextualRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, texts, embeddings, llm):
        super().__init__(texts, embeddings, llm)
    def retrieve(self,query,k=3,user_context=None):
        print("retrieving contextual")
        # Use LLM to incorporate user context into the query
        context_prompt = PromptTemplate(
            input_variables=["query", "context"],
            template="Given the user context: {context}\nReformulate the query to best address the user's needs: {query}"
        )
        context_chain= context_prompt | self.llm
        input_data={"query":query,"context":user_context or "No specific context provided"}
        contextualized_query=context_chain.invoke(input_data).content
        print(f'contextualized query: {contextualized_query}')

        # Retrieve documents using the contextualized query
        docs=self.bd.similarity_search(contextualized_query,k=k*2)

        # Use LLM to rank the relevance of retrieved documents considering the user context
        ranking_prompt = PromptTemplate(
            input_variables=["query", "context", "doc"],
            template="Given the query: '{query}' and user context: '{context}', rate the relevance of this document on a scale of 1-10:\nDocument: {doc}\nRelevance score:"
        )
        ranking_chain = ranking_prompt | self.llm.with_structured_output(relevant_score)
        print("ranking docs")
        ranked_docs=[]
        for doc in docs:
            input_data={"query":query,"context":user_context or "No specific context provided","doc":doc.page_content}
            score=float ( ranking_chain.invoke(input_data).score)
            ranked_docs.append((doc,score))
        ranked_docs.sort(key=lambda x : x[1],reverse=True)

        return [doc for doc, _ in ranked_docs[:k]]


##Define the Adapive retriever class

In [62]:
class AdaptiveRetriever:
    def __init__(self,texts,embeddings,llm):
        self.classifier=QueryClassifier()
        self.strategies = {
            "Factual": FactualRetrievalStrategy(texts, embeddings, llm),
            "Analytical": AnalyticalRetrievalStrategy(texts, embeddings, llm),
            "Opinion": OpinionRetrievalStrategy(texts, embeddings, llm),
            "Contextual": ContextualRetrievalStrategy(texts, embeddings, llm)
        }
    def get_relevant_documents(self, query: str) -> List[Document]:
        category = self.classifier.classify(query)
        strategy = self.strategies[category]
        return strategy.retrieve(query)

##Define aditional retriever that inherits from langchain BaseRetriever
這是為了能夠完美融入 LangChain 的生態系統中

牆壁插座 (LangChain Ecosystem): 預期插入的是標準的 BaseRetriever 規格。

你的電器 (AdaptiveRetriever): 功能強大，但接頭規格是自定義的（普通的 Python Class）。

轉接頭 (PydanticAdaptiveRetriever): 把你的電器轉接成牆壁插座能接受的形狀。

In [63]:
class PydanticAdaptiveRetriever(BaseRetriever):
    adaptive_retriever: AdaptiveRetriever = Field(exclude=True)

    class Config:
        arbitrary_types_allowed = True
    def _get_relevant_documents(self, query: str) -> List[Document]:
        return self.adaptive_retriever.get_relevant_documents(query)
    def get_relevant_documents(self, query) -> List[Document]:
        return self.adaptive_retriever.get_relevant_documents(query)
    async def aget_relevant_documents(self, query: str) -> List[Document]:
        return self.get_relevant_documents(query)

/tmp/ipython-input-764268965.py:1: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class PydanticAdaptiveRetriever(BaseRetriever):


##Define the Adaptive RAG class

In [64]:
class AdaptiveRAG:
    def __init__(self,texts:List[str],embeddings,llm):
        adaptive_retriever = AdaptiveRetriever(texts,embeddings,llm)
        self.retriever = PydanticAdaptiveRetriever(adaptive_retriever=adaptive_retriever)
        self.llm=llm
        prompt_template = """Use the following pieces of context to answer the question at the end.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.

        {context}

        Question: {query}
        Answer:"""

        prompt=PromptTemplate.from_template(prompt_template)

        self.llm_chain=prompt | self.llm

    def answer(self,query):
        docs=self.retriever.get_relevant_documents(query)
        input_data={"context":"\n".join([doc.page_content for doc in docs]),"query":query}
        return self.llm_chain.invoke(input_data)

##假想文章與範例

In [65]:

texts=["""
    Large language models (LLMs) inevitably
exhibit hallucinations since the accuracy of
generated texts cannot be secured solely by
the parametric knowledge they encapsulate. Although
retrieval-augmented generation (RAG)
is a practicable complement to LLMs, it relies
heavily on the relevance of retrieved documents,
raising concerns about how the model
behaves if retrieval goes wrong. To this end, we
propose the Corrective Retrieval Augmented
Generation (CRAG) to improve the robustness
of generation. Specifically, a lightweight
retrieval evaluator is designed to assess the
overall quality of retrieved documents for a
query, returning a confidence degree based
on which different knowledge retrieval actions
can be triggered. Since retrieval from
static and limited corpora can only return suboptimal
documents, large-scale web searches
are utilized as an extension for augmenting the
retrieval results. Besides, a decompose-thenrecompose
algorithm is designed for retrieved
documents to selectively focus on key information
and filter out irrelevant information in
them. CRAG is plug-and-play and can be
seamlessly coupled with various RAG-based
approaches. Experiments on four datasets
covering short- and long-form generation tasks
show that CRAG can significantly improve the
performance of RAG-based approaches. """
]

rag_system = AdaptiveRAG(texts,embeddings,llm)

In [32]:
! ollama list

NAME                           ID              SIZE      MODIFIED      
qwen3:4b-instruct-2507-q8_0    aa7252f68dda    4.3 GB    3 minutes ago    


In [66]:
!ps aux | grep ollama

root        1274  3.5  0.4 2500860 65412 ?       Sl   12:58   0:36 ollama serve
root        5771  0.0  0.0   7376  3580 ?        S    13:15   0:00 /bin/bash -c ps aux | grep ollama
root        5773  0.0  0.0   6484  2564 ?        S    13:15   0:00 grep ollama


In [67]:
factual_result = rag_system.answer("What exactly did the CRAG propose to improve?").content
print(f"\nAnswer: {factual_result}")

query classify
Factual
retrieving factual
enchance query:Certainly! Here's an enhanced version of your factual query, optimized for better information retrieval:

"What specific infrastructure, policy, or technological improvements did the CRAG (Community Resilience and Advancement Group) propose to address key societal or environmental challenges, and what were the primary goals behind these recommendations?"

This refined query:
- Clarifies the subject (CRAG) and its likely scope.
- Specifies the type of improvements (infrastructure, policy, technology).
- Adds context by referencing the challenges they aimed to address.
- Asks for the underlying goals, which helps retrieve more comprehensive and meaningful responses.

Note: If CRAG refers to a specific known organization (e.g., in a particular region or field), that detail should be included for even greater precision.
ranking docs

Answer: CRAG proposed to improve the robustness of retrieval-augmented generation (RAG) by introducin

In [69]:
analytical_result = rag_system.answer("How would CRAG's proposed improvements impact its goals?").content
print(f"\nAnswer: {analytical_result}")

query classify
Analytical

Answer: CRAG's proposed improvements—such as using a lightweight retrieval evaluator to assess document quality, triggering appropriate retrieval actions based on confidence degrees, extending retrieval with large-scale web searches, and employing a decompose-then-recompose algorithm to focus on key information and filter out irrelevant content—directly enhance its core goals. These improvements increase the robustness of generation by ensuring that only high-quality, relevant retrieved information is used. By dynamically adjusting retrieval strategies based on confidence and incorporating broader, more diverse sources when needed, CRAG reduces the risk of hallucinations and improves the accuracy and reliability of generated outputs. Additionally, its plug-and-play design allows seamless integration with existing RAG-based approaches, making it a practical and effective enhancement for real-world applications. Overall, these features collectively improve the 

In [70]:


opinion_result = rag_system.answer("What are different perspectives on CRAG's proposed improvements?").content
print(f"Answer: {opinion_result}")



query classify
Contextual
retrieving contextual
contextualized query: What are the various viewpoints on CRAG's proposed improvements?
ranking docs
Answer: The provided context describes CRAG as a method designed to improve the robustness of retrieval-augmented generation (RAG) by introducing a lightweight retrieval evaluator that assesses the quality of retrieved documents and triggers different knowledge retrieval actions based on confidence degrees. It also mentions that large-scale web searches are used to extend retrieval results and that a decompose-then-recompose algorithm helps focus on key information and filter out irrelevant content. Experiments on four datasets show that CRAG can significantly improve the performance of RAG-based approaches.

However, the context does not present or discuss different perspectives (such as criticisms, alternative viewpoints, or debates) on CRAG's proposed improvements. Therefore, based on the given information, there are no different perspec

In [71]:
contextual_result = rag_system.answer("How does CRAG's proposal fit into the broader context of its mission?").content
print(f"Answer: {contextual_result}")

query classify
Contextual
retrieving contextual
contextualized query: How does CRAG's proposed initiative align with and advance its overall mission and strategic goals?
ranking docs
Answer: CRAG's proposal fits into the broader context of its mission by enhancing the robustness and reliability of retrieval-augmented generation (RAG) systems. Since large language models (LLMs) are prone to hallucinations and RAG depends heavily on the relevance and quality of retrieved documents, CRAG introduces a corrective mechanism through a lightweight retrieval evaluator that assesses the quality of retrieved content and assigns a confidence degree. Based on this confidence, the system can trigger different retrieval actions, such as performing large-scale web searches to supplement static, limited corpora with more relevant and up-to-date information. Additionally, CRAG employs a decompose-then-recompose algorithm to selectively focus on key information and filter out irrelevant content, improvin